# Financial Growth & Target Planner

In [7]:
import plotly.graph_objs as go
import ipywidgets as widgets
from IPython.display import display, HTML
from datetime import datetime


# --- Title and description as HTML widget ---
title_desc_html = widgets.HTML(
    value="""
    <h1 style="text-align:center; margin-bottom:10px;">Investment Dashboard</h1>
    <p style="text-align:left; font-size:16px; margin-top:0;">
        This dashboard simulates investing in a portfolio over multiple years. 
        You start with an initial amount and contribute a monthly savings, 
        while the portfolio grows with a specified annual interest rate. 
        Set a target amount to see how long it will take to reach your goal, 
        taking bavarian taxes into account (26.375% consisting of 25% Kapitalerstragssteuer + 5.5% solidarity surcharge on the capital tax).
    </p>
    """,
    layout=widgets.Layout(width='100%')
)

# --- Functions to calculate wealth over time ---
def wealth_over_time(start_amount, annual_interest, monthly_payment, years, tax_rate_percent):
    tax_rate = tax_rate_percent / 100
    months = years * 12
    monthly_interest = (1 + annual_interest / 100) ** (1/12) - 1
    amount = start_amount
    amounts = [start_amount]
    net_amounts = [start_amount]
    for m in range(1, months + 1):
        amount = amount * (1 + monthly_interest) + monthly_payment
        total_payments = start_amount + monthly_payment * m
        gross_profit = amount - total_payments
        taxes_paid = max(0, gross_profit * tax_rate)
        net_amount = amount - taxes_paid
        amounts.append(amount)
        net_amounts.append(net_amount)
    return amounts, net_amounts

def format_euro(amount):
    return f"{int(round(amount)):,}".replace(",", " ")

# --- Update plot function ---
def update_plot(start, interest, monthly_savings, years, target, tax_rate_percent):
    tax_rate = tax_rate_percent / 100
    amounts, net_amounts = wealth_over_time(start, interest, monthly_savings, years, tax_rate_percent)
    months_list = list(range(len(amounts)))
    current_year = datetime.now().year
    years_labels = [str(current_year + m // 12) for m in months_list]
    year_ticks = [i for i in range(len(months_list)) if (i % 12 == 0)]
    year_labels = [years_labels[i] for i in year_ticks]

    # Calculate summary results
    final_amount = amounts[-1]
    gross_profit = final_amount - (start + monthly_savings * years * 12)
    taxes_paid = max(0, gross_profit * tax_rate)
    net_profit = gross_profit - taxes_paid

    # Calculate years to target (gross)
    def years_to_target(start, interest, monthly_savings, target):
        months = 0
        monthly_interest = (1 + interest / 100) ** (1/12) - 1
        amount = start
        while amount < target:
            amount = amount * (1 + monthly_interest) + monthly_savings
            months += 1
        return months // 12, months % 12

    # Calculate years to target (net)
    def years_to_target_after_tax(start, interest, monthly_savings, target, tax_rate):
        months = 0
        monthly_interest = (1 + interest / 100) ** (1/12) - 1
        amount = start
        while True:
            amount = amount * (1 + monthly_interest) + monthly_savings
            months += 1
            total_payments = start + monthly_savings * months
            gross_profit = amount - total_payments
            taxes_paid = max(0, gross_profit * tax_rate)
            net_amount = amount - taxes_paid
            if net_amount >= target:
                break
        return months // 12, months % 12

    years_needed, months_needed = years_to_target(start, interest, monthly_savings, target)
    years_needed_net, months_needed_net = years_to_target_after_tax(start, interest, monthly_savings, target, tax_rate)

    # --- Prepare result HTML ---
    result_text = (f"<b>Results</b><br>"
                   f"Final amount: {format_euro(final_amount)}€<br>"
                   f"Gross profit: {format_euro(gross_profit)}€<br>"
                   f"Net profit: {format_euro(net_profit)}€<br>"
                   f"Taxes paid: {format_euro(taxes_paid)}€<br>"
                   f"Time to reach target: {years_needed_net} years, {months_needed_net} months")

    # --- Display widgets and results side by side ---
    widgets_box.layout.width = '50%'
    results_box = widgets.HTML(value=f'<div style="text-align:left;">{result_text}</div>', layout=widgets.Layout(width='50%'))
    display(widgets.HBox([widgets_box, results_box], layout=widgets.Layout(width='100%')))

    # --- Plotly graph full width ---
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=months_list, y=amounts, mode='lines', name='Gross Wealth'))
    fig.add_trace(go.Scatter(x=months_list, y=net_amounts, mode='lines', name='Net Wealth (after taxes)'))
    fig.update_layout(
        title='Portfolio Growth Over Time',
        xaxis=dict(title='Year', tickmode='array', tickvals=year_ticks, ticktext=year_labels),
        yaxis=dict(title='Wealth (€)', range=[0, None]),
        legend_title='Legend',
        template='plotly_white',
        autosize=True,
        width=None,
        height=500,
        margin=dict(t=80, l=20, r=20, b=40)
    )
    display(fig)

# --- Widgets ---
start_widget = widgets.IntText(value=60000, description='Start (€):')
interest_widget = widgets.FloatText(value=10, description='Interest (%):')
monthly_savings_widget = widgets.IntText(value=1500, description='Monthly (€):')
years_widget = widgets.IntText(value=10, description='Years:')
target_widget = widgets.IntText(value=500000, description='Target (€):')
tax_rate_widget = widgets.FloatText(value=26.375, description='Tax Rate (%):')

widgets_box = widgets.VBox([start_widget, interest_widget, monthly_savings_widget, years_widget, target_widget, tax_rate_widget])

# --- Interactive output ---
out = widgets.interactive_output(update_plot, {
    'start': start_widget,
    'interest': interest_widget,
    'monthly_savings': monthly_savings_widget,
    'years': years_widget,
    'target': target_widget,
    'tax_rate_percent': tax_rate_widget,
})

# --- Final dashboard layout as a VBox to integrate everything ---
dashboard_vbox = widgets.VBox([title_desc_html, out], layout=widgets.Layout(width='100%', padding='10px'))
display(dashboard_vbox)
